## Cleaning BLS county unemployment rates data

#### Initial importing

In [1]:
import pandas as pd
import numpy as np

This txt file is from https://data.bls.gov/cgi-bin/srgate

In [2]:
df = pd.read_csv('County unemployment rates 1990-2023.txt', low_memory=False)
df.head()

,Series ID,Jan 1990,Feb 1990,Mar 1990,Apr 1990,May 1990,Jun 1990,Jul 1990,Aug 1990,Sep 1990,...,Mar 2023,Apr 2023,May 2023,Jun 2023,Jul 2023,Aug 2023,Sep 2023,Oct 2023,Nov 2023,Dec 2023
0,LAUCN010010000000003,6.5,6.5,5.7,6.6,6.0,7.1,6.0,6.7,7.3,...,1.7,1.6,1.9,2.3,2.0,2.2,2.2,2.1,2.2,2.2(P)
1,LAUCN010030000000003,6.4,6.1,5.3,4.8,4.3,4.8,4.9,4.9,5.3,...,1.8,1.6,1.9,2.3,2.0,2.2,2.2,2.1,2.3,2.5(P)
2,LAUCN010050000000003,7.2,7.2,6.3,7.5,7.6,9.0,7.9,9.0,9.0,...,3.5,3.3,3.7,4.9,4.1,4.3,4.0,4.1,4.2,4.5(P)
3,LAUCN010070000000003,12.6,9.7,5.4,5.5,4.8,6.2,5.6,15.8,9.1,...,1.7,1.6,2.2,2.9,2.4,2.5,2.4,2.3,2.5,2.4(P)
4,LAUCN010090000000003,5.3,5.9,10.9,5.6,5.3,5.8,5.7,6.5,6.3,...,1.6,1.5,1.9,2.3,2.0,2.1,2.2,2.0,2.1,2.3(P)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3224 entries, 0 to 3223
Columns: 409 entries, Series ID to Dec 2023
dtypes: object(409)
memory usage: 10.1+ MB


### Cleaning goals

When this csv was created at https://data.bls.gov/cgi-bin/srgate, this warning was created:\
\
P : Preliminary.\
U : The annual average cannot be calculated due to missing monthly data.\
Y : Data reflect controlling to interpolated statewide totals because the survey was not conducted.\
N : Not available.\
\
These codes are causing the data to be interpeted as objects, rather than integers. They must be removed.\
\
For P, I will simply remove it. The data can be updated later, as needed.\
U does not appear in the data, as annual averages were not included.\
For Y, I will simply remove it. It's a good enough estimate and doesn't need to be marked for my purposes.\
For N, for now I will replace it with NaN. Ideally these NaNs will be replaced with estimates later.\
\
This was not stated in the warning, but counties that did not exist at a given time have their unemployment rate marked as ' ' (a single space).\
I could mark these as NaNs, but as they should not be replaced with estimates later on, I will instead simply drop these rows.\
\
Furthermore, the county names and FIPS county codes should be added.\
I would also prefer the data be in long format, with one column for time and one for the rate of unemployment during that time.\
Finally, I would like to convert the new time column to datetime.

#### Adding county names

county_names.csv adapted from: https://www.bls.gov/help/hlpforma.htm

In [4]:
names = pd.read_csv('county_names.csv')
names.head()

,area_text,series_ID
0,"Autauga County, AL",LAUCN010010000000003
1,"Baldwin County, AL",LAUCN010030000000003
2,"Barbour County, AL",LAUCN010050000000003
3,"Bibb County, AL",LAUCN010070000000003
4,"Blount County, AL",LAUCN010090000000003


In [5]:
df = pd.merge(left=df, right=names, left_on='Series ID', right_on='series_ID')
df.head()

,Series ID,Jan 1990,Feb 1990,Mar 1990,Apr 1990,May 1990,Jun 1990,Jul 1990,Aug 1990,Sep 1990,...,May 2023,Jun 2023,Jul 2023,Aug 2023,Sep 2023,Oct 2023,Nov 2023,Dec 2023,area_text,series_ID
0,LAUCN010010000000003,6.5,6.5,5.7,6.6,6.0,7.1,6.0,6.7,7.3,...,1.9,2.3,2.0,2.2,2.2,2.1,2.2,2.2(P),"Autauga County, AL",LAUCN010010000000003
1,LAUCN010030000000003,6.4,6.1,5.3,4.8,4.3,4.8,4.9,4.9,5.3,...,1.9,2.3,2.0,2.2,2.2,2.1,2.3,2.5(P),"Baldwin County, AL",LAUCN010030000000003
2,LAUCN010050000000003,7.2,7.2,6.3,7.5,7.6,9.0,7.9,9.0,9.0,...,3.7,4.9,4.1,4.3,4.0,4.1,4.2,4.5(P),"Barbour County, AL",LAUCN010050000000003
3,LAUCN010070000000003,12.6,9.7,5.4,5.5,4.8,6.2,5.6,15.8,9.1,...,2.2,2.9,2.4,2.5,2.4,2.3,2.5,2.4(P),"Bibb County, AL",LAUCN010070000000003
4,LAUCN010090000000003,5.3,5.9,10.9,5.6,5.3,5.8,5.7,6.5,6.3,...,1.9,2.3,2.0,2.1,2.2,2.0,2.1,2.3(P),"Blount County, AL",LAUCN010090000000003


In [6]:
df = df.drop('Series ID', axis=1)
df.head()

,Jan 1990,Feb 1990,Mar 1990,Apr 1990,May 1990,Jun 1990,Jul 1990,Aug 1990,Sep 1990,Oct 1990,...,May 2023,Jun 2023,Jul 2023,Aug 2023,Sep 2023,Oct 2023,Nov 2023,Dec 2023,area_text,series_ID
0,6.5,6.5,5.7,6.6,6.0,7.1,6.0,6.7,7.3,7.2,...,1.9,2.3,2.0,2.2,2.2,2.1,2.2,2.2(P),"Autauga County, AL",LAUCN010010000000003
1,6.4,6.1,5.3,4.8,4.3,4.8,4.9,4.9,5.3,4.9,...,1.9,2.3,2.0,2.2,2.2,2.1,2.3,2.5(P),"Baldwin County, AL",LAUCN010030000000003
2,7.2,7.2,6.3,7.5,7.6,9.0,7.9,9.0,9.0,8.3,...,3.7,4.9,4.1,4.3,4.0,4.1,4.2,4.5(P),"Barbour County, AL",LAUCN010050000000003
3,12.6,9.7,5.4,5.5,4.8,6.2,5.6,15.8,9.1,9.2,...,2.2,2.9,2.4,2.5,2.4,2.3,2.5,2.4(P),"Bibb County, AL",LAUCN010070000000003
4,5.3,5.9,10.9,5.6,5.3,5.8,5.7,6.5,6.3,6.3,...,1.9,2.3,2.0,2.1,2.2,2.0,2.1,2.3(P),"Blount County, AL",LAUCN010090000000003


#### Long format

In [7]:
df = pd.melt(df, id_vars=['series_ID','area_text'], value_vars=list(df.columns)[:-2])
df

,series_ID,area_text,variable,value
0,LAUCN010010000000003,"Autauga County, AL",Jan 1990,6.5
1,LAUCN010030000000003,"Baldwin County, AL",Jan 1990,6.4
2,LAUCN010050000000003,"Barbour County, AL",Jan 1990,7.2
3,LAUCN010070000000003,"Bibb County, AL",Jan 1990,12.6
4,LAUCN010090000000003,"Blount County, AL",Jan 1990,5.3
...,...,...,...,...
1315387,LAUCN721450000000003,"Vega Baja Municipio, PR",Dec 2023,7.0(P)
1315388,LAUCN721470000000003,"Vieques Municipio, PR",Dec 2023,4.8(P)
1315389,LAUCN721490000000003,"Villalba Municipio, PR",Dec 2023,10.1(P)
1315390,LAUCN721510000000003,"Yabucoa Municipio, PR",Dec 2023,8.1(P)


In [8]:
df.columns = ['series_ID', 'county_state','date','unemployment_rate']
df.head()

,series_ID,county_state,date,unemployment_rate
0,LAUCN010010000000003,"Autauga County, AL",Jan 1990,6.5
1,LAUCN010030000000003,"Baldwin County, AL",Jan 1990,6.4
2,LAUCN010050000000003,"Barbour County, AL",Jan 1990,7.2
3,LAUCN010070000000003,"Bibb County, AL",Jan 1990,12.6
4,LAUCN010090000000003,"Blount County, AL",Jan 1990,5.3


#### Removing code letters and changing to float

In [9]:
df['unemployment_rate'] = df.unemployment_rate.str.replace('(P)' , '')
df.tail()

,series_ID,county_state,date,unemployment_rate
1315387,LAUCN721450000000003,"Vega Baja Municipio, PR",Dec 2023,7.0
1315388,LAUCN721470000000003,"Vieques Municipio, PR",Dec 2023,4.8
1315389,LAUCN721490000000003,"Villalba Municipio, PR",Dec 2023,10.1
1315390,LAUCN721510000000003,"Yabucoa Municipio, PR",Dec 2023,8.1
1315391,LAUCN721530000000003,"Yauco Municipio, PR",Dec 2023,10.5


In [10]:
df['unemployment_rate'] = df.unemployment_rate.str.replace('(Y)' , '')
df.head()

,series_ID,county_state,date,unemployment_rate
0,LAUCN010010000000003,"Autauga County, AL",Jan 1990,6.5
1,LAUCN010030000000003,"Baldwin County, AL",Jan 1990,6.4
2,LAUCN010050000000003,"Barbour County, AL",Jan 1990,7.2
3,LAUCN010070000000003,"Bibb County, AL",Jan 1990,12.6
4,LAUCN010090000000003,"Blount County, AL",Jan 1990,5.3


In [11]:
df['unemployment_rate'] = df.unemployment_rate.replace('-(N)', np.nan)
df.head()

,series_ID,county_state,date,unemployment_rate
0,LAUCN010010000000003,"Autauga County, AL",Jan 1990,6.5
1,LAUCN010030000000003,"Baldwin County, AL",Jan 1990,6.4
2,LAUCN010050000000003,"Barbour County, AL",Jan 1990,7.2
3,LAUCN010070000000003,"Bibb County, AL",Jan 1990,12.6
4,LAUCN010090000000003,"Blount County, AL",Jan 1990,5.3


In [12]:
df = df[df.unemployment_rate != ' ']

In [13]:
df = df.astype({'unemployment_rate': 'Float64'})

#### Brief examination of the data

In [14]:
df.describe()

,unemployment_rate
count,1312646.0
mean,6.101084
std,3.409005
min,0.0
25%,3.8
50%,5.3
75%,7.5
max,58.7


I find the existence of 0.0% unemployment suspicious enough to investigate.

In [15]:
df[df.unemployment_rate == 0.0]

,series_ID,county_state,date,unemployment_rate
12349,LAUCN483010000000003,"Loving County, TX",Apr 1990,0.0
15578,LAUCN483110000000003,"McMullen County, TX",May 1990,0.0
25250,LAUCN483110000000003,"McMullen County, TX",Aug 1990,0.0
28474,LAUCN483110000000003,"McMullen County, TX",Sep 1990,0.0
31698,LAUCN483110000000003,"McMullen County, TX",Oct 1990,0.0
38141,LAUCN483010000000003,"Loving County, TX",Dec 1990,0.0
41365,LAUCN483010000000003,"Loving County, TX",Jan 1991,0.0
44589,LAUCN483010000000003,"Loving County, TX",Feb 1991,0.0
47813,LAUCN483010000000003,"Loving County, TX",Mar 1991,0.0
51037,LAUCN483010000000003,"Loving County, TX",Apr 1991,0.0


It appears that only two very small counties are recorded as ever having 0.0% unemployment, and only for a fairly short time. This seems believable enough.

#### Convert to datetime

In [16]:
df['date'] = pd.to_datetime(df['date'], format='%b %Y')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1312872 entries, 0 to 1315391
Data columns (total 4 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   series_ID          1312872 non-null  object        
 1   county_state       1312872 non-null  object        
 2   date               1312872 non-null  datetime64[ns]
 3   unemployment_rate  1312646 non-null  Float64       
dtypes: Float64(1), datetime64[ns](1), object(2)
memory usage: 51.3+ MB


#### Adding FIPS county codes

Tableau works best with FIPS codes. These are conveniently contained within the series ID as the first five numerals.

In [17]:
df['FIPS'] = df.series_ID.str[5:10]
df

,series_ID,county_state,date,unemployment_rate,FIPS
0,LAUCN010010000000003,"Autauga County, AL",1990-01-01,6.5,01001
1,LAUCN010030000000003,"Baldwin County, AL",1990-01-01,6.4,01003
2,LAUCN010050000000003,"Barbour County, AL",1990-01-01,7.2,01005
3,LAUCN010070000000003,"Bibb County, AL",1990-01-01,12.6,01007
4,LAUCN010090000000003,"Blount County, AL",1990-01-01,5.3,01009
...,...,...,...,...,...
1315387,LAUCN721450000000003,"Vega Baja Municipio, PR",2023-12-01,7.0,72145
1315388,LAUCN721470000000003,"Vieques Municipio, PR",2023-12-01,4.8,72147
1315389,LAUCN721490000000003,"Villalba Municipio, PR",2023-12-01,10.1,72149
1315390,LAUCN721510000000003,"Yabucoa Municipio, PR",2023-12-01,8.1,72151


#### Split county_name to county and state columns, add DC as state for District of Columbia, optionally remove "County" from end of each county name.

In [18]:
df[['county', 'state']] = df['county_state'].str.split(', ', n=1, expand=True)
df

,series_ID,county_state,date,unemployment_rate,FIPS,county,state
0,LAUCN010010000000003,"Autauga County, AL",1990-01-01,6.5,01001,Autauga County,AL
1,LAUCN010030000000003,"Baldwin County, AL",1990-01-01,6.4,01003,Baldwin County,AL
2,LAUCN010050000000003,"Barbour County, AL",1990-01-01,7.2,01005,Barbour County,AL
3,LAUCN010070000000003,"Bibb County, AL",1990-01-01,12.6,01007,Bibb County,AL
4,LAUCN010090000000003,"Blount County, AL",1990-01-01,5.3,01009,Blount County,AL
...,...,...,...,...,...,...,...
1315387,LAUCN721450000000003,"Vega Baja Municipio, PR",2023-12-01,7.0,72145,Vega Baja Municipio,PR
1315388,LAUCN721470000000003,"Vieques Municipio, PR",2023-12-01,4.8,72147,Vieques Municipio,PR
1315389,LAUCN721490000000003,"Villalba Municipio, PR",2023-12-01,10.1,72149,Villalba Municipio,PR
1315390,LAUCN721510000000003,"Yabucoa Municipio, PR",2023-12-01,8.1,72151,Yabucoa Municipio,PR


In [19]:
df = df.fillna(value={'state':'DC'})

In [20]:
#df_long['county'] = df_long.county.str.replace(' County/city' , '')
#df_long['county'] = df_long.county.str.replace(' County/town' , '')
#df_long['county'] = df_long.county.str.replace(' County', '')

#### Swapping FIPS

Tableau doesn't accept some historical FIPS, so here I swap them for their nearest modern counterpart.

In [21]:
FIPS_swapper = {'02201':'02198',
'02232':'02105',
'02280':'02195'}
for old_FIPS, new_FIPS in FIPS_swapper.items():
    df['FIPS'] = df.FIPS.replace(old_FIPS, new_FIPS)

#### Final cleaning: remove unneeded column, reordering columns

In [22]:
df = df.drop('series_ID', axis=1)
df = df[['FIPS','date','county','state','county_state','unemployment_rate']]
df.head()

,FIPS,date,county,state,county_state,unemployment_rate
0,01001,1990-01-01,Autauga County,AL,"Autauga County, AL",6.5
1,01003,1990-01-01,Baldwin County,AL,"Baldwin County, AL",6.4
2,01005,1990-01-01,Barbour County,AL,"Barbour County, AL",7.2
3,01007,1990-01-01,Bibb County,AL,"Bibb County, AL",12.6
4,01009,1990-01-01,Blount County,AL,"Blount County, AL",5.3


#### Exporting to csv

In [23]:
#df.to_csv('county_unemployment_1990-2023-cleaned_2.csv', index=False)

#### From monthly to annual data

In [24]:
df = df.groupby(['FIPS',df.date.dt.year,'county','state','county_state'])['unemployment_rate'].mean().reset_index()

In [25]:
df.date = df.date.astype(str)+'-07-01'
df.unemployment_rate = round(df.unemployment_rate,1)
df

,FIPS,date,county,state,county_state,unemployment_rate
0,01001,1990-07-01,Autauga County,AL,"Autauga County, AL",6.5
1,01001,1991-07-01,Autauga County,AL,"Autauga County, AL",6.7
2,01001,1992-07-01,Autauga County,AL,"Autauga County, AL",6.5
3,01001,1993-07-01,Autauga County,AL,"Autauga County, AL",5.9
4,01001,1994-07-01,Autauga County,AL,"Autauga County, AL",5.0
...,...,...,...,...,...,...
109401,72153,2019-07-01,Yauco Municipio,PR,"Yauco Municipio, PR",14.6
109402,72153,2020-07-01,Yauco Municipio,PR,"Yauco Municipio, PR",12.8
109403,72153,2021-07-01,Yauco Municipio,PR,"Yauco Municipio, PR",11.7
109404,72153,2022-07-01,Yauco Municipio,PR,"Yauco Municipio, PR",10.3


#### Swapping more FIPS

As I continued working with Tableau, I discovered there were more FIPS that weren't accepted, so I swap them here. I also swap some names and merge some data to improve consistency. See here for details: https://github.com/spearitual/Updated-FIPS-codes-for-Tableau 


In [26]:
FIPS_swapper = {'51540': '51003',
 '51580': '51005',
 '51820': '51015',
 '51680': '51031',
 '51640': '51035',
 '51730': '51053',
 '51610': '51059',
 '51840': '51069',
 '51595': '51081',
 '51690': '51089',
 '51830': '51095',
 '15005': '15009',
 '51750': '51121',
 '51590': '51143',
 '51670': '51149',
 '51685': '51153',
 '51775': '51161',
 '51678': '51163',
 '51660': '51165',
 '51620': '51175',
 '51630': '51177',
 '51520': '51191',
 '51720': '51195',
 '51735': '51199',
 '51790': '51015',
 '51570': '51053',
 '51600': '51059',
 '51683': '51153',
 '51530': '51163',
 '09013':'09170',
 '09001':'09190',}
name_swapper = {'Charlottesville city': 'Albemarle + Charlottesville',
 'Covington city': 'Alleghany + Covington',
 'Waynesboro city': 'Augusta, Staunton + Waynesboro',
 'Lynchburg city': 'Campbell + Lynchburg',
 'Galax city': 'Carroll + Galax',
 'Petersburg city': 'Dinwiddie, Colonial Heights + Petersburg',
 'Falls Church city': 'Fairfax, Fairfax City + Falls Church',
 'Winchester city': 'Frederick + Winchester',
 'Emporia city': 'Greensville + Emporia',
 'Martinsville city': 'Henry + Martinsville',
 'Williamsburg city': 'James City + Williamsburg',
 'Kalawao County': 'Maui + Kalawao',
 'Radford city': 'Montgomery + Radford',
 'Danville city': 'Pittsylvania + Danville',
 'Hopewell city': 'Prince George + Hopewell',
 'Manassas Park city': 'Prince William, Manassas + Manassas Park',
 'Salem city': 'Roanoke + Salem',
 'Lexington city': 'Rockbridge, Buena Vista + Lexington',
 'Harrisonburg city': 'Rockingham + Harrisonburg',
 'Franklin city': 'Southampton + Franklin',
 'Fredericksburg city': 'Spotsylvania + Fredericksburg',
 'Bristol city': 'Washington + Bristol',
 'Norton city': 'Wise + Norton',
 'Poquoson city': 'York + Poquoson',
 'Staunton city': 'Augusta, Staunton + Waynesboro',
 'Colonial Heights city': 'Dinwiddie, Colonial Heights + Petersburg',
 'Fairfax city': 'Fairfax, Fairfax City + Falls Church',
 'Manassas city': 'Prince William, Manassas + Manassas Park',
 'Buena Vista city': 'Rockbridge, Buena Vista + Lexington',
 'Albemarle County': 'Albemarle + Charlottesville',
 'Alleghany County': 'Alleghany + Covington',
 'Augusta County': 'Augusta, Staunton + Waynesboro',
 'Campbell County': 'Campbell + Lynchburg',
 'Carroll County': 'Carroll + Galax',
 'Dinwiddie County': 'Dinwiddie, Colonial Heights + Petersburg',
 'Fairfax County': 'Fairfax, Fairfax City + Falls Church',
 'Frederick County': 'Frederick + Winchester',
 'Greensville County': 'Greensville + Emporia',
 'Henry County': 'Henry + Martinsville',
 'James City County': 'James City + Williamsburg',
 'Maui County': 'Maui + Kalawao',
 'Montgomery County': 'Montgomery + Radford',
 'Pittsylvania County': 'Pittsylvania + Danville',
 'Prince George County': 'Prince George + Hopewell',
 'Prince William County': 'Prince William, Manassas + Manassas Park',
 'Roanoke County': 'Roanoke + Salem',
 'Rockbridge County': 'Rockbridge, Buena Vista + Lexington',
 'Rockingham County': 'Rockingham + Harrisonburg',
 'Southampton County': 'Southampton + Franklin',
 'Spotsylvania County': 'Spotsylvania + Fredericksburg',
 'Washington County': 'Washington + Bristol',
 'Wise County': 'Wise + Norton',
 'York County': 'York + Poquoson'}

In [27]:
for old_FIPS, new_FIPS in FIPS_swapper.items():
    df['FIPS'] = df.FIPS.replace(old_FIPS, new_FIPS)
for old_name, new_name in name_swapper.items():
    df['county'] = df.county.replace(old_name, new_name)

In [28]:
df['county_state'] = df.county + ', ' + df.state
df = df.rename(columns={'state':'state_abbr'})
df = df[['FIPS','date','county','state_abbr','county_state','unemployment_rate']]

In [29]:
df = df.groupby(['FIPS','date','county','state_abbr','county_state']).agg('mean').reset_index()
#Averaging the means will be imprecise, but good enough.
#It would be better to compare labor force numbers of the county and city and use that to come up with a combined rate.

Here I make a few last swaps for the counties that only began to exist in 2020.

In [30]:
FIPS_swapper = {'09005':'09160',
               '09009':'09140',
               '09003':'09110',
               '09007':'09130',
               '09015':'09150',
               '09011':'09180'}

In [31]:
for old_FIPS, new_FIPS in FIPS_swapper.items():
    df.loc[(df['date'] >= "2020-07-01") & (df['FIPS'] == old_FIPS), "FIPS"] = new_FIPS

#### Export to csv

In [32]:
#df.to_csv('county_unemployment_1990-2023_annual_2.csv', index=False)